# 文本分类实例

## 第一步：导入相关包

In [8]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification,Trainer,TrainingArguments
from datasets import load_dataset

## 第二步：加载数据集

In [9]:
dataset = load_dataset('dirtycomputer/ChnSentiCorp_htl_all',split='train')
dataset = dataset.filter(lambda x:x['review'] is not None)


## 第三步：划分数据集

In [10]:
datasets = dataset.train_test_split(test_size=0.1)

## 第四步：数据集预处理

In [11]:
tokenizer = AutoTokenizer.from_pretrained('hfl/rbt3')

def process_function(examples):
    tokenized_exexamples = tokenizer(examples['review'],max_length=128,truncation=True)
    tokenized_exexamples['labels'] = examples['label']
    return tokenized_exexamples

tokenizer_datasets = datasets.map(process_function,batched=True,remove_columns=datasets['train'].column_names)
    

Map: 100%|██████████████████████████| 777/777 [00:00<00:00, 7199.86 examples/s]
